# Exercise 3

In [ ]:
import numpy
from IPython.display import Image
from matplotlib import pyplot
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.tools.visualization import plot_bloch_multivector
from qiskit.tools.visualization import plot_histogram
from qiskit import execute
from qiskit import BasicAer

# Specify the statevector simulator backend to be used for circuit simulation
# We specify two different backends for generating two types of plots:

# Histogram
backend_hist = BasicAer.get_backend('qasm_simulator')

# Bloch spheres
backend_bloch = BasicAer.get_backend('statevector_simulator')

# We include the Bloch sphere backend for visualisation purposes alongside its use for
# finding the state vector of the circuit output

*Run this cell to obtain latex ket command*
$$\newcommand{\ket}[1]{\lvert{#1}\rangle}$$

## Quantum Teleportation

An interesting use for quantum computing and information is for its applications to cryptography. The phenomena of quantum teleportation provides a method by which we can transfer information losslessly between two parties whilst ensuring that no useful data can be intercepted by an eavesdropper.

Implementing this in a quantum circuit is achieved through imposing a special state of entanglement between three qubits that allows for the state of one qubit to be transferred to another without it collapsing. The nature of this superposition causes a random rotation of the transferred state into one of four possible outputs with equal probability. This rotation is mirrored in the two qubits belonging to the sender, and so measuring the final state of these two qubits in the computational basis tells us what rotations must be made to the received qubit so as to return to the desired state. 

It is important to note that this limits the speed at which useful communication between sender and receiver can occur; since whilst the state itself may be instantaneously transferred, classical information must still be sent about the measurements of the senders qubits in order for the receiver to obtain the correct state. It is, however, also the reason why quantum communication is so secure; an eavesdropper can only ever intercept information about the required corrections, whereas the coefficients that characterize the information within the system have been instantaneously teleported without the necessity of sending the information across space, thus bypassing any potential eavesdropping.

Note that we will collapse the final state of our "teleported" qubit for measurement purposes, but in principle the state could be preserved and used as the input for another quantum process once the relevant corrections have been made so as to return to the desired state. 

We have provided the code for these corrections and the mathematical blueprint for construction of the quantum teleportation system has been provided to you in the handout accompanying this task aswell as in pdf format (Teleportation.pdf).

In this exercise you must build a circuit to perform this quantum teleportation process, sending the following state between two qubits: 

$$
    \ket{\psi} = \frac{\sqrt{3}}{2}\ket{0} + \frac{1}{2}\ket{1}
$$

The measurements are to be plotted on a histogram and you should use the maths in the handout to justify why your results match expectation.

In [ ]:
# Create a three qubit register and three classical registers to hold qubit measurements
q = QuantumRegister(3, 'q') 
c0 = ClassicalRegister(1, 'c0')
c1 = ClassicalRegister(1, 'c1')
c2 = ClassicalRegister(1, 'c2')

# Note: the sender has the qubit to teleport (qubit 0) and one an entangled qubit (qubit 1)
# The receiver has the other entangled qubit (qubit 2)

# Initialise circuit
circuit = QuantumCircuit(q, c0,c1,c2)

# Generate state to teleport
circuit.ry(numpy.pi/3, q[0])

# The gate 'ry' is a rotation operator around the y axis of the bloch sphere (not Hilbert space) 

# Entangle qubit 1 and 2 in the beta_00 Bell state
circuit.h(q[1])
circuit.cx(q[1], q[2])

# Place barrier to control sequence of operations
circuit.barrier(q)

# Entangle the senders qubits
circuit.cx(q[0], q[1])

# Measure qubit 1 in the computational basis
circuit.measure(q[1], c1[0])

# Measure qubit 0 in the +/- basis
circuit.h(q[0])
circuit.measure(q[0], c0[0])

# Place barrier to control sequence of operations
circuit.barrier(q)

# Apply relevant correction based on measurement of senders qubits
circuit.z(q[2]).c_if(c0, 1)
circuit.x(q[2]).c_if(c1, 1)

# Measure the state of the teleported qubit to check results    
circuit.measure(q[2], c2[0])

# Draw the circuit diagram
circuit.draw(output="mpl")

In [ ]:
# Create a Quantum Program for execution 
job_hist = execute(circuit, backend_hist, shots=5000)
job_bloch = execute(circuit, backend_bloch)

# Obtain results
result_hist = job_hist.result()
result_bloch = job_bloch.result()

In [ ]:
# Plot a histogram of the obtained measurements
plot_histogram(result_hist.get_counts(circuit))